In [2]:
mkdir -p ~/.kaggle



SyntaxError: invalid syntax (<ipython-input-2-dbd3af7ca7bb>, line 1)

In [3]:
mkdir -p ~/.kaggle

In [4]:
cp kaggle.json ~/.kaggle/

In [7]:
!chmod 600 ~/.kaggle/kaggle.json



In [8]:
!kaggle datasets download -d saurabhshahane/ecommerce-text-classification

 64% 5.00M/7.86M [00:00<00:00, 31.3MB/s]
100% 7.86M/7.86M [00:00<00:00, 46.6MB/s]


In [9]:
!unzip /content/ecommerce-text-classification.zip


Archive:  /content/ecommerce-text-classification.zip
  inflating: ecommerceDataset.csv    


In [11]:
import pandas as pd

In [13]:
dset=pd.read_csv("/content/ecommerceDataset.csv",names=["category","description"],header=None)

In [14]:
dset.shape

(50425, 2)

In [15]:
dset.head()

,category,description
0,Household,Paper Plane Design Framed Wall Hanging Motivat...
1,Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,Household,Incredible Gifts India Wooden Happy Birthday U...


In [16]:
dset.dropna(inplace=True)

In [17]:
dset

,category,description
0,Household,Paper Plane Design Framed Wall Hanging Motivat...
1,Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,Household,Incredible Gifts India Wooden Happy Birthday U...
...,...,...
50420,Electronics,Strontium MicroSD Class 10 8GB Memory Card (Bl...
50421,Electronics,CrossBeats Wave Waterproof Bluetooth Wireless ...
50422,Electronics,Karbonn Titanium Wind W4 (White) Karbonn Titan...
50423,Electronics,"Samsung Guru FM Plus (SM-B110E/D, Black) Colou..."


In [18]:
dset.category.unique()

array(['Household', 'Books', 'Clothing & Accessories', 'Electronics'],
      dtype=object)

In [19]:
dset.describe()

,category,description
count,50424,50424
unique,4,27802
top,Household,Think & Grow Rich About the Author NAPOLEON HI...
freq,19313,30


In [20]:
dset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50424 entries, 0 to 50424
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   category     50424 non-null  object
 1   description  50424 non-null  object
dtypes: object(2)
memory usage: 1.2+ MB


In [23]:
# when you train the fasttext model you need to specify the label

dset['category']='__label__'+dset['category'].astype(str)

In [24]:
dset.head(5)

,category,description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...


In [26]:
dset['category_description']=dset['category']+' '+dset['description']

In [27]:
dset.head()

,category,description,category_description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...,__label__Household Paper Plane Design Framed W...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",__label__Household SAF 'Floral' Framed Paintin...
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...,__label__Household SAF 'UV Textured Modern Art...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1...",__label__Household SAF Flower Print Framed Pai...
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...,__label__Household Incredible Gifts India Wood...


In [28]:
import re

In [30]:
text="VIKI'S | Bookcase/Bookshelf (3-Shelf/Shelve,White) | ? . hi"
text=re.sub(r'[^\w\s\']',' ',text)
text=re.sub(' +',' ',text)
text.strip().lower()

"viki's bookcase bookshelf 3 shelf shelve white hi"

In [32]:
def preprocess(text):
  text = re.sub(r'[^\w\s\']',' ', text)
  text=re.sub(' +',' ',text)
  return text.strip().lower()

In [33]:
dset['category_description']=dset['category_description'].map(preprocess)

In [34]:
dset.head()

,category,description,category_description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...,__label__household paper plane design framed w...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",__label__household saf 'floral' framed paintin...
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...,__label__household saf 'uv textured modern art...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1...",__label__household saf flower print framed pai...
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...,__label__household incredible gifts india wood...


In [35]:
from sklearn.model_selection import train_test_split
train,test=train_test_split(dset,test_size=0.2)

In [36]:
train.shape,test.shape

((40339, 3), (10085, 3))

In [38]:
train.to_csv("ecommerce.train", columns=["category_description"], index=False, header=False)
test.to_csv("ecommerce.test", columns=["category_description"], index=False, header=False)

In [41]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199773 sha256=337254754e7128a09e595e03287b0d6ad4b9a010b6a76ce745c37478037226f4
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [42]:
import fasttext

In [43]:
model=fasttext.train_supervised(input="ecommerce.train")
model.test("ecommerce.test")

(10084, 0.9731257437524792, 0.9731257437524792)

In [44]:
model.predict("This laptop has a fast processor and a sleek design")

(('__label__electronics',), array([0.98010176]))

In [46]:
model.predict("i want to buy a machine for cutting the wire")

(('__label__books',), array([0.67183936]))

In [47]:
model.predict("i want a short ")

(('__label__books',), array([1.00000978]))

In [48]:
model.predict("i want a t short that made from cotton ")

(('__label__books',), array([0.99833494]))

In [50]:
model.predict("ckey men's cotton t shirt fabric details 80 cotton 20 polyester super combed cotton rich fabric")

(('__label__clothing',), array([0.99904108]))

In [51]:
model.get_nearest_neighbors('painting')

[(0.9987289309501648, 'phdstore'),
 (0.9987174868583679, 'collander'),
 (0.9987102150917053, 'hd2583'),
 (0.9987065196037292, 'bombino'),
 (0.9987053275108337, '65x80cm'),
 (0.9986969232559204, 'hl7576'),
 (0.9986953139305115, 'approx64'),
 (0.9986943602561951, 'insulator'),
 (0.9986896514892578, 'mochen'),
 (0.9986896514892578, 'comtributes')]